In [1]:
# Libraries for reading, cleaning and plotting the dataa
import numpy as np 
import pandas as pd 
import os

# Libraries for models 
from sklearn.linear_model import LinearRegression


In [2]:
def linear_model():
    '''This function generates a linear model to predict change in sea level rise in cm (global mean) given historical co2 emissions in gt'''
    # Read in training data 
    df= pd.read_excel("slr_data.xlsx",sheet_name="data")
    #add a variable for change in sea level
    sl_chg =np.diff(df["sea_level (cm)"].to_numpy())
    #get the x variable without final year
    co2_gt = df["co2_gt"][:-1]
    #create a new df for co2 emission(x) and change in sea level (y)
    new_df = pd.DataFrame(zip(sl_chg,co2_gt),columns=["sl_chg","co2_gt"])

    #split the data into training and dev datasets
    training_df = new_df.sample(frac=0.8)
    dev_df = new_df.drop(training_df.index)

    train_data = training_df.co2_gt.to_numpy().reshape(-1,1)
    train_label = training_df["sl_chg"]

    dev_data = dev_df.co2_gt.to_numpy().reshape(-1,1)
    dev_label = dev_df["sl_chg"]
    
    #fit a linear model
    model = LinearRegression()
    model.fit(train_data,train_label)

#     #score the model
    print("R2: ", model.score(dev_data,dev_label))
    return model

In [3]:
def gen_slr(gt):
    '''This function takes a new co2 emission in gt and gives slr in cm'''
    if gt <=0:
        return 0
    else:
        model=linear_model()
        #predicts global mean sea level in cm.
        return model.predict(np.array([gt]).reshape(-1,1))

In [4]:
gen_slr(30)

R2:  -0.3914689846398576


array([0.38455535])